<a href="https://colab.research.google.com/github/pabvald/Vision-Project-Image-Segmentation/blob/pablo-task1/Vision_task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Segmentation Task 1
#### Welcome to the first task of Image Segmentation. Image segmentation is the process of partitioning the image into a set of pixels representing an object. In this task, you will be introduced to the problem of image segmentation and programming pipeline involved in image segmentation.

For the purpose of this task we will be using PASCAL VOC datset. The dataset contains a total of 2913 images with segmentation annotations. Code in the cell below will download the code and extract the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar -xvf VOCtrainval_11-May-2012.tar

In [2]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 110kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


### 1.1 Loading the dataset

In [1]:
import os
import copy
import time
from os.path import join as pjoin
import collections
import json
import torch
import imageio
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt
import glob

from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import transforms
from torchvision import models

In [2]:
%config Completer.use_jedi = False
torch.manual_seed(0)
np.random.seed(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  """Entry point for launching an IPython kernel.


#### pascalVOCDataset

In [3]:
class pascalVOCDataset(data.Dataset):
    """Data loader for the Pascal VOC semantic segmentation dataset.

    Annotations from both the original VOC data (which consist of RGB images
    in which colours map to specific classes) and the SBD (Berkely) dataset
    (where annotations are stored as .mat files) are converted into a common
    `label_mask` format.  Under this format, each mask is an (M,N) array of
    integer values from 0 to 21, where 0 represents the background class.

    The label masks are stored in a new folder, called `pre_encoded`, which
    is added as a subdirectory of the `SegmentationClass` folder in the
    original Pascal VOC data layout.

    A total of five data splits are provided for working with the VOC data:
        train: The original VOC 2012 training data - 1464 images
        val: The original VOC 2012 validation data - 1449 images
        trainval: The combination of `train` and `val` - 2913 images
        train_aug: The unique images present in both the train split and
                   training images from SBD: - 8829 images (the unique members
                   of the result of combining lists of length 1464 and 8498)
        train_aug_val: The original VOC 2012 validation data minus the images
                   present in `train_aug` (This is done with the same logic as
                   the validation set used in FCN PAMI paper, but with VOC 2012
                   rather than VOC 2011) - 904 images
    """

    def __init__(
        self,
        root,
        sbd_path=None,
        split="train_aug",
        is_transform=False,
        img_size=512,
        augmentations=None,
        img_norm=True,
        test_mode=False,
    ):
        self.root = root
        self.sbd_path = sbd_path
        self.split = split
        self.is_transform = is_transform
        self.augmentations = augmentations
        self.img_norm = img_norm
        self.test_mode = test_mode
        self.n_classes = 21
        self.mean = np.array([104.00699, 116.66877, 122.67892])
        self.files = collections.defaultdict(list)
        self.img_size = img_size if isinstance(img_size, tuple) else (img_size, img_size)

        if not self.test_mode:
            for split in ["train", "val", "trainval"]:
                path = pjoin(self.root, "ImageSets/Segmentation", split + ".txt")
                file_list = tuple(open(path, "r"))
                file_list = [id_.rstrip() for id_ in file_list]
                self.files[split] = file_list
            self.setup_annotations()

        self.tf = transforms.Compose(
            [
                # add more trasnformations as you see fit
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    def __len__(self):
        return len(self.files[self.split])

    def __getitem__(self, index):
        im_name = self.files[self.split][index]
        im_path = pjoin(self.root, "JPEGImages", im_name + ".jpg")
        lbl_path = pjoin(self.root, "SegmentationClass/pre_encoded", im_name + ".png")
        im = Image.open(im_path)
        lbl = Image.open(lbl_path)
        if self.augmentations is not None:
            im, lbl = self.augmentations(im, lbl)
        if self.is_transform:
            im, lbl = self.transform(im, lbl)
        return im, torch.clamp(lbl, max=20)

    def transform(self, img, lbl):
        if self.img_size == ("same", "same"):
            pass
        else:
            img = img.resize((self.img_size[0], self.img_size[1]))  # uint8 with RGB mode
            lbl = lbl.resize((self.img_size[0], self.img_size[1]))
        img = self.tf(img)
        lbl = torch.from_numpy(np.array(lbl)).long()
        lbl[lbl == 255] = 0
        return img, lbl

    def get_pascal_labels(self):
        """Load the mapping that associates pascal classes with label colors

        Returns:
            np.ndarray with dimensions (21, 3)
        """
        return np.asarray(
            [
                [0, 0, 0],
                [128, 0, 0],
                [0, 128, 0],
                [128, 128, 0],
                [0, 0, 128],
                [128, 0, 128],
                [0, 128, 128],
                [128, 128, 128],
                [64, 0, 0],
                [192, 0, 0],
                [64, 128, 0],
                [192, 128, 0],
                [64, 0, 128],
                [192, 0, 128],
                [64, 128, 128],
                [192, 128, 128],
                [0, 64, 0],
                [128, 64, 0],
                [0, 192, 0],
                [128, 192, 0],
                [0, 64, 128],
            ]
        )

    def encode_segmap(self, mask):
        """Encode segmentation label images as pascal classes

        Args:
            mask (np.ndarray): raw segmentation label image of dimension
              (M, N, 3), in which the Pascal classes are encoded as colours.

        Returns:
            (np.ndarray): class map with dimensions (M,N), where the value at
            a given location is the integer denoting the class index.
        """
        mask = mask.astype(int)
        label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
        for ii, label in enumerate(self.get_pascal_labels()):
            label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
        label_mask = label_mask.astype(int)
        # print(np.unique(label_mask))
        return label_mask

    def decode_segmap(self, label_mask, plot=False):
        """Decode segmentation class labels into a color image

        Args:
            label_mask (np.ndarray): an (M,N) array of integer values denoting
              the class label at each spatial location.
            plot (bool, optional): whether to show the resulting color image
              in a figure.

        Returns:
            (np.ndarray, optional): the resulting decoded color image.
        """
        label_colours = self.get_pascal_labels()
        r = label_mask.copy()
        g = label_mask.copy()
        b = label_mask.copy()
        for ll in range(0, self.n_classes):
            r[label_mask == ll] = label_colours[ll, 0]
            g[label_mask == ll] = label_colours[ll, 1]
            b[label_mask == ll] = label_colours[ll, 2]
        rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
        rgb[:, :, 0] = r / 255.0
        rgb[:, :, 1] = g / 255.0
        rgb[:, :, 2] = b / 255.0
        if plot:
            plt.imshow(rgb)
            plt.show()
        else:
            return rgb

    def setup_annotations(self):
        """Sets up Berkley annotations by adding image indices to the
        `train_aug` split and pre-encode all segmentation labels into the
        common label_mask format (if this has not already been done). This
        function also defines the `train_aug` and `train_aug_val` data splits
        according to the description in the class docstring
        """
        sbd_path = self.sbd_path
        target_path = pjoin(self.root, "SegmentationClass/pre_encoded")
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        train_aug = self.files["train"]

        # keep unique elements (stable)
        train_aug = [train_aug[i] for i in sorted(np.unique(train_aug, return_index=True)[1])]
        self.files["train_aug"] = train_aug
        set_diff = set(self.files["val"]) - set(train_aug)  # remove overlap
        self.files["train_aug_val"] = list(set_diff)

        pre_encoded = glob.glob(pjoin(target_path, "*.png"))
        expected = np.unique(self.files["train_aug"] + self.files["val"]).size

        if len(pre_encoded) != expected:
            print("Pre-encoding segmentation masks...")

            for ii in tqdm(self.files["trainval"]):
                fname = ii + ".png"
                lbl_path = pjoin(self.root, "SegmentationClass", fname)
                lbl = self.encode_segmap(m.imread(lbl_path))
                lbl = m.toimage(lbl, high=lbl.max(), low=lbl.min())
                m.imsave(pjoin(target_path, fname), lbl)

        assert expected == 2913, "unexpected dataset sizes"

### 1.2 Define the model architecture(2.0 point)
In this section you have the freedom to decide your own model. Keep in mind though, to perform image segmentation, you need to implement an architecture that does pixel level classification i.e. for each pixel in the image you need to predict the probability of it belonging to one of the 21 categories.

#### SegNet-Basic class

In [7]:
 INPUT_CHANNELS = 3 
 OUTPUT_CHANNELS = 21 #Number of classes

 ENCODER_DIMS = {
      'block00': { 'in': [INPUT_CHANNELS, 64],'out': [64, 64] },
      'block01': { 'in': [64, 128],       'out': [128, 128] },
      'block02': { 'in': [128, 256, 256], 'out': [256, 256, 256] },
      'block03': { 'in': [256, 512, 512], 'out': [512, 512, 512] },
    }

 DECODER_DIMS = {
      'block04': { 'in': [512, 512, 512], 'out': [512, 512, 512] },
      'block03': { 'in': [512, 512, 512], 'out': [512, 512, 256] },
      'block02': { 'in': [256, 256, 256], 'out': [256, 256, 128] },
      'block01': { 'in': [128, 128],       'out': [128, OUTPUT_CHANNELS] },
    }

In [4]:
def normal_init(m, mean=0, std=1):
  """Initialize the weights of a layer usinga normal distribution 
    
    Ref: https://discuss.pytorch.org/t/pytorch-how-to-initialize-weights/81511/2
  """
  if isinstance(m, (nn.Linear, nn.Conv2d)):
      m.weight.data.normal_(mean, std)
      if m.bias.data is not None:
          m.bias.data.zero_()
  elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
      m.weight.data.fill_(1)
      if m.bias.data is not None:
          m.bias.data.zero_()


class conv2DBatchNormRelu(nn.Module):
  """ Conv2d - Batch Normalization - ReLU module 
    
    Ref: https://www.programmersought.com/article/28751510513/
  """

  def __init__(self, in_channels, out_channels, kernel_size, stride, padding,
                  bias=True, dilation=1, is_batchnorm=True):
    super(conv2DBatchNormRelu, self).__init__()
    if is_batchnorm:
        self.cbr_unit = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                      bias=bias, dilation=dilation),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    else:
        self.cbr_unit = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                      bias=bias, dilation=dilation),
            nn.ReLU(inplace=True)
        )
  
  def forward(self, input):
      output = self.cbr_unit(input)
      return output


class convTranspose2DBatchNorm(nn.Module):
  """ ConvTranspose2d - Batch Normalization module """

  def __init__(self, in_channels, out_channels, kernel_size, stride, padding,
                  bias=True, dilation=1, is_batchnorm=True):
    super(convTranspose2DBatchNorm, self).__init__()
    if is_batchnorm:
        self.cbr_unit = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, 
                                padding=padding, bias=bias, dilation=dilation),
            nn.BatchNorm2d(out_channels)
        )
    else:
        self.cbr_unit = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                      bias=bias, dilation=dilation)
        )
  
  def forward(self, input):
      output = self.cbr_unit(input)
      return output


class segnetBasicEncoderBlock(nn.Module):
  """Segnet-Basic Encoder Block module"""

  def __init__(self, in_channels, out_channels):
    super(segnetBasicEncoderBlock, self).__init__()
    self.conv1 = conv2DBatchNormRelu(in_channels, out_channels, 7, 1, 3, bias=False)
    self.conv2 = conv2DBatchNormRelu(out_channels, out_channels, 7, 1, 3, bias=False)
    self.conv3 = conv2DBatchNormRelu(out_channels, out_channels, 7, 1, 3, bias=False)
    self.conv4 = conv2DBatchNormRelu(out_channels, out_channels, 7, 1, 3, bias=False)
    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)

  def forward(self, input):
    output = self.conv1(input)
    output = self.conv2(output)
    output = self.conv3(output)
    output = self.conv4(output)
    unpooled_size = output.size()
    outputs, indices = self.maxpool(output)
    return outputs, indices, unpooled_size


class segnetBasicDecoderBlock(nn.Module):
  """Segnet-Basic Decoder Block module"""
  def __init__(self, in_channels, out_channels):
    super(segnetBasicDecoderBlock, self).__init__()
    self.unpool = nn.MaxUnpool2d(2,2)
    self.convTrans1 = convTranspose2DBatchNorm(in_channels, out_channels, 7, 1, 3, bias=False)
    self.convTrans2 = convTranspose2DBatchNorm(out_channels, out_channels, 7, 1, 3, bias=False)
    self.convTrans3 = convTranspose2DBatchNorm(out_channels, out_channels, 7, 1, 3, bias=False)
    self.convTrans4 = convTranspose2DBatchNorm(out_channels, out_channels, 7, 1, 3, bias=False)

  def forward(self, input, indices, output_size):
    output = self.unpool(input, indices=indices, output_size=output_size)
    output = self.convTrans1(output)
    output = self.convTrans2(output)
    output = self.convTrans3(output)
    output = self.convTrans4(output)
    return output


class SegnetBasic1(nn.Module):
  """SegNet Class"""

  def __init__(self, input_channels=3, output_channels=21):
    """Initialize an instance of SegNet

      Args:
        input_channels (int): number of input channels 
        output_channels (int): number of output channels
    """
    super(SegnetBasic1, self).__init__()
    self.enc1 = segnetBasicEncoderBlock(input_channels, 64)
    self.dec1 = segnetBasicDecoderBlock(64, output_channels)
    self.softmax = nn.Softmax(dim=1)
    self.debug = False   

    self.weight_init()

  def forward(self, x): 
    enc1, indices1, unpool_size1 = self.enc1(x)

    dec1 = self.dec1(enc1, indices=indices1, output_size=unpool_size1)
    output = self.softmax(dec1)

    return output

  def weight_init(self):
    for block in self._modules:
        try:
            for m in self._modules[block]:
                normal_init(m)
        except:
            normal_init(block)

In [16]:
model = SegnetBasic1().cuda()

In [17]:
local_path = 'VOCdevkit/VOC2012/' # modify it according to your device
bs = 5
epochs =  10

### 1.4 Dataset and Dataloader(0.5 points)
Create the dataset using pascalVOCDataset class defined above. Use local_path defined in the cell above as root. 

In [18]:
# dataset variable
training_data = pascalVOCDataset(local_path, split="train", is_transform=True)
validation_data = pascalVOCDataset(local_path, split="val", is_transform=True)

print("Training data instances: ", len(training_data))
print("Validation data instances:", len(validation_data))

Training data instances:  1464
Validation data instances: 1449


In [19]:
dataloaders = {
    'train' : data.DataLoader(training_data, batch_size=bs, shuffle=True, num_workers=0),
    'val': data.DataLoader(validation_data, batch_size=bs, shuffle=True, num_workers=0)
}

### 1.5 Loss fuction and Optimizer(1.0 point)
Define below with the loss function you think would be most suitable for segmentation task. You are free to choose any optimizer to train the network.

In [20]:
# loss function
loss_f = nn.CrossEntropyLoss()

# optimizer variable
opt = optim.LBFGS(model.parameters())

### 1.6 Training the model(3.0 points)
Your task here is to complete the code below to perform a training loop and save the model weights after each epoch of training.

In [10]:
input, labels = next(iter(dataloaders['train']))
input = input.cuda()
input.size()

torch.Size([10, 3, 512, 512])

In [12]:
output = model.forward(input)

In [20]:
output.size()

torch.Size([10, 21, 512, 512])

In [21]:
since = time.time()

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(epochs):
    print('Epoch {}/{}'.format(epoch, epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.cuda()
            labels = labels.cuda()

            # zero the parameter gradients
            opt.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = loss_f(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    opt.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            phase, epoch_loss, epoch_acc))

        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))
print('Best val Acc: {:4f}'.format(best_acc))

# load best model weights
model.load_state_dict(best_model_wts)

Epoch 0/9
----------


RuntimeError: ignored

### 1.7 Evaluate your model(1.5 points)
In this section you have to implement the evaluation metrics for your model. Calculate the values of F1-score, dice coefficient and AUC-ROC score on the data you used for training. You can use external packages like scikit-learn to compute above metrics.

In [ ]:
def evaluate(ground_truth, predictions):
    
    return f1_score, auc_score, dice_coeeficient

### 1.8 Plot the evaluation metrics against epochs(1.0)
In section 1.6 we saved the weights of the model after each epoch. In this section, you have to calculate the evaluation metrics after each epoch of training by loading the weights for each epoch. Once you have calculated the evaluation metrics for each epoch, plot them against the epochs.

### 1.9 Visualize results(0.5 points)
For any 10 images in the dataset, show the images along the with their segmentation mask.